In [1]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import numpy as np

In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [8]:
df = pd.read_csv('SIAPE_remuneracao_liquida_executivo_sexo_raca.csv')
df

,ano,liquido,sexo,raca
0,1999,"6527,2",Masculino,Branco
1,1999,"4768,7",Masculino,Negro
2,1999,"5261,6",Feminino,Branco
3,1999,4010,Feminino,Negro
4,2000,"5690,3",Masculino,Branco
...,...,...,...,...
83,2019,"5996,2",Feminino,Negro
84,2020,"8774,2",Masculino,Branco
85,2020,"6272,2",Masculino,Negro
86,2020,"7753,8",Feminino,Branco


In [9]:
df['sexo_raca']= df['sexo'] + ' ' + df['raca'] #tratamento
df

,ano,liquido,sexo,raca,sexo_raca
0,1999,"6527,2",Masculino,Branco,Masculino Branco
1,1999,"4768,7",Masculino,Negro,Masculino Negro
2,1999,"5261,6",Feminino,Branco,Feminino Branco
3,1999,4010,Feminino,Negro,Feminino Negro
4,2000,"5690,3",Masculino,Branco,Masculino Branco
...,...,...,...,...,...
83,2019,"5996,2",Feminino,Negro,Feminino Negro
84,2020,"8774,2",Masculino,Branco,Masculino Branco
85,2020,"6272,2",Masculino,Negro,Masculino Negro
86,2020,"7753,8",Feminino,Branco,Feminino Branco


In [10]:
df.rename(columns={'liquido': 'remuneracao_liquida', 'raca' : 'cor_raca'}, inplace= True) #padronizando


In [11]:
df

,ano,remuneracao_liquida,sexo,cor_raca,sexo_raca
0,1999,"6527,2",Masculino,Branco,Masculino Branco
1,1999,"4768,7",Masculino,Negro,Masculino Negro
2,1999,"5261,6",Feminino,Branco,Feminino Branco
3,1999,4010,Feminino,Negro,Feminino Negro
4,2000,"5690,3",Masculino,Branco,Masculino Branco
...,...,...,...,...,...
83,2019,"5996,2",Feminino,Negro,Feminino Negro
84,2020,"8774,2",Masculino,Branco,Masculino Branco
85,2020,"6272,2",Masculino,Negro,Masculino Negro
86,2020,"7753,8",Feminino,Branco,Feminino Branco


In [12]:
df['remuneracao_liquida'] = df['remuneracao_liquida'].str.replace(',','.').astype(float)


In [13]:
df

,ano,remuneracao_liquida,sexo,cor_raca,sexo_raca
0,1999,6527.2,Masculino,Branco,Masculino Branco
1,1999,4768.7,Masculino,Negro,Masculino Negro
2,1999,5261.6,Feminino,Branco,Feminino Branco
3,1999,4010.0,Feminino,Negro,Feminino Negro
4,2000,5690.3,Masculino,Branco,Masculino Branco
...,...,...,...,...,...
83,2019,5996.2,Feminino,Negro,Feminino Negro
84,2020,8774.2,Masculino,Branco,Masculino Branco
85,2020,6272.2,Masculino,Negro,Masculino Negro
86,2020,7753.8,Feminino,Branco,Feminino Branco


In [14]:
df = df[['ano', 'sexo', 'cor_raca', 'sexo_raca', 'remuneracao_liquida' ]]
df

,ano,sexo,cor_raca,sexo_raca,remuneracao_liquida
0,1999,Masculino,Branco,Masculino Branco,6527.2
1,1999,Masculino,Negro,Masculino Negro,4768.7
2,1999,Feminino,Branco,Feminino Branco,5261.6
3,1999,Feminino,Negro,Feminino Negro,4010.0
4,2000,Masculino,Branco,Masculino Branco,5690.3
...,...,...,...,...,...
83,2019,Feminino,Negro,Feminino Negro,5996.2
84,2020,Masculino,Branco,Masculino Branco,8774.2
85,2020,Masculino,Negro,Masculino Negro,6272.2
86,2020,Feminino,Branco,Feminino Branco,7753.8


Subindo pro GBQ

In [15]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

c:\Users\ana.sales_republica\Anaconda\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [16]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
 bigquery.SchemaField('sexo', 'STRING', description= 'Sexo da pessoa observada'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('sexo_raca', 'STRING', description= 'Sexo e raça da pessoa observada'),
 bigquery.SchemaField('remuneracao_liquida','FLOAT',description='Remuneração liquida da pessoa observada')]

In [17]:
table_ref = dataset_ref.table('SIAPE_remuneracao_liquida_executivo_sexo_raca')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref,job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=9f74d901-c33c-49bb-9c07-fcec49ef2d06>